### Frame number count between manual (fixed video) vs CV2 (fixed video) vs CV2 count (original video) for PS1

In [ ]:
import cv2
import os
import concurrent.futures
import logging

# Setup basic configuration for logging
logging.basicConfig(filename='video_processing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def count_frames(video_path):
    """
    Count frames manually and compare with cv2's reported frame count.
    Returns the video path, reported count, and manually counted frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Failed to open {video_path}")
        return None, None, None
    
    reported_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    manual_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        manual_frame_count += 1

    cap.release()
    return video_path, reported_frame_count, manual_frame_count

def process_video_pair(video1_path, video2_path):
    """
    Process a pair of video files and compare frame counts.
    """
    path1, reported1, manual1 = count_frames(video1_path)
    path2, reported2, manual2 = count_frames(video2_path)

    # Compare and collect mismatches
    if reported1 != manual1 or reported2 != manual2 or manual1 != manual2:
        return {
            'video1': {'path': path1, 'reported': reported1, 'manual': manual1},
            'video2': {'path': path2, 'reported': reported2, 'manual': manual2}
        }
    return None

def process_and_compare(directory1, directory2):
    """
    Process video files in both directories and compare frame counts.
    """
    video_files1 = [f for f in os.listdir(directory1) if f.endswith(('.avi', '.mp4', '.mov'))]
    video_files2 = [f for f in os.listdir(directory2) if f.endswith('.mp4')]

    mismatches = []
    video_pairs = []

    for video_file in video_files1:
        base_filename = video_file.rsplit('.', 1)[0]  # Remove extension for matching
        matching_file = base_filename + '_fixed.mp4'  # Construct the fixed filename

        if matching_file in video_files2:
            video_pairs.append((os.path.join(directory1, video_file),
                                os.path.join(directory2, matching_file)))

    # Process video pairs in parallel using ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_video_pair, *pair): pair for pair in video_pairs}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                mismatches.append(result)
            # Log progress
            video_pair = futures[future]
            logging.info(f"Processed pair: {video_pair[0]}, {video_pair[1]}")

    if mismatches:
        for mismatch in mismatches:
            logging.info("Mismatch found:")
            logging.info(f"Video 1: {mismatch['video1']['path']}, Reported: {mismatch['video1']['reported']}, Manual: {mismatch['video1']['manual']}")
            logging.info(f"Video 2: {mismatch['video2']['path']}, Reported: {mismatch['video2']['reported']}, Manual: {mismatch['video2']['manual']}")
    else:
        logging.info("No mismatches found.")

if __name__ == "__main__":
    directory1_path ="/storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1"
    directory2_path = "/storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed"
    process_and_compare(directory1_path, directory2_path)

2024-04-17 22:47:56,996 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_023_V4_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed/R01_023_V4_PS1_fixed.mp4
2024-04-17 22:50:53,378 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_006_V4_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed/R01_006_V4_PS1_fixed.mp4
2024-04-17 22:54:27,483 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_019_V4_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed/R01_019_V4_PS1_fixed.mp4
2024-04-17 22:55:01,675 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_026_V5_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fi

2024-04-17 23:42:31,507 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_048_V5_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed/R01_048_V5_PS1_fixed.mp4
2024-04-17 23:43:08,532 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_074_V3_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed/R01_074_V3_PS1_fixed.mp4
2024-04-17 23:43:50,930 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_089_V3_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS1_vids_fixed/R01_089_V3_PS1_fixed.mp4
2024-04-17 23:44:40,899 - INFO - Processed pair: /storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize1/R01_009_V5_PS1.avi, /storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fi

### Frame number count between manual (fixed video) vs CV2 (fixed video) vs CV2 count (original video) for PS2

In [ ]:
import cv2
import os
import concurrent.futures
import logging

# Setup basic configuration for logging
logging.basicConfig(filename='video_processing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def count_frames(video_path):
    """
    Count frames manually and compare with cv2's reported frame count.
    Returns the video path, reported count, and manually counted frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Failed to open {video_path}")
        return None, None, None
    
    reported_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    manual_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        manual_frame_count += 1

    cap.release()
    return video_path, reported_frame_count, manual_frame_count

def process_video_pair(video1_path, video2_path):
    """
    Process a pair of video files and compare frame counts.
    """
    path1, reported1, manual1 = count_frames(video1_path)
    path2, reported2, manual2 = count_frames(video2_path)

    # Compare and collect mismatches
    if reported1 != manual1 or reported2 != manual2 or manual1 != manual2:
        return {
            'video1': {'path': path1, 'reported': reported1, 'manual': manual1},
            'video2': {'path': path2, 'reported': reported2, 'manual': manual2}
        }
    return None

def process_and_compare(directory1, directory2):
    """
    Process video files in both directories and compare frame counts.
    """
    video_files1 = [f for f in os.listdir(directory1) if f.endswith(('.avi', '.mp4', '.mov'))]
    video_files2 = [f for f in os.listdir(directory2) if f.endswith('.mp4')]

    mismatches = []
    video_pairs = []

    for video_file in video_files1:
        base_filename = video_file.rsplit('.', 1)[0]  # Remove extension for matching
        matching_file = base_filename + '_fixed.mp4'  # Construct the fixed filename

        if matching_file in video_files2:
            video_pairs.append((os.path.join(directory1, video_file),
                                os.path.join(directory2, matching_file)))

    # Process video pairs in parallel using ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_video_pair, *pair): pair for pair in video_pairs}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                mismatches.append(result)
            # Log progress
            video_pair = futures[future]
            logging.info(f"Processed pair: {video_pair[0]}, {video_pair[1]}")

    if mismatches:
        for mismatch in mismatches:
            logging.info("Mismatch found:")
            logging.info(f"Video 1: {mismatch['video1']['path']}, Reported: {mismatch['video1']['reported']}, Manual: {mismatch['video1']['manual']}")
            logging.info(f"Video 2: {mismatch['video2']['path']}, Reported: {mismatch['video2']['reported']}, Manual: {mismatch['video2']['manual']}")
    else:
        logging.info("No mismatches found.")

if __name__ == "__main__":
    directory1_path ="/storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize2"
    directory2_path = "/storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS2_vids_fixed"
    process_and_compare(directory1_path, directory2_path)

### Frame number count between manual (fixed video) vs CV2 (fixed video) vs CV2 count (original video) for PS3

In [ ]:
import cv2
import os
import concurrent.futures
import logging

# Setup basic configuration for logging
logging.basicConfig(filename='video_processing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def count_frames(video_path):
    """
    Count frames manually and compare with cv2's reported frame count.
    Returns the video path, reported count, and manually counted frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Failed to open {video_path}")
        return None, None, None
    
    reported_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    manual_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        manual_frame_count += 1

    cap.release()
    return video_path, reported_frame_count, manual_frame_count

def process_video_pair(video1_path, video2_path):
    """
    Process a pair of video files and compare frame counts.
    """
    path1, reported1, manual1 = count_frames(video1_path)
    path2, reported2, manual2 = count_frames(video2_path)

    # Compare and collect mismatches
    if reported1 != manual1 or reported2 != manual2 or manual1 != manual2:
        return {
            'video1': {'path': path1, 'reported': reported1, 'manual': manual1},
            'video2': {'path': path2, 'reported': reported2, 'manual': manual2}
        }
    return None

def process_and_compare(directory1, directory2):
    """
    Process video files in both directories and compare frame counts.
    """
    video_files1 = [f for f in os.listdir(directory1) if f.endswith(('.avi', '.mp4', '.mov'))]
    video_files2 = [f for f in os.listdir(directory2) if f.endswith('.mp4')]

    mismatches = []
    video_pairs = []

    for video_file in video_files1:
        base_filename = video_file.rsplit('.', 1)[0]  # Remove extension for matching
        matching_file = base_filename + '_fixed.mp4'  # Construct the fixed filename

        if matching_file in video_files2:
            video_pairs.append((os.path.join(directory1, video_file),
                                os.path.join(directory2, matching_file)))

    # Process video pairs in parallel using ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_video_pair, *pair): pair for pair in video_pairs}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                mismatches.append(result)
            # Log progress
            video_pair = futures[future]
            logging.info(f"Processed pair: {video_pair[0]}, {video_pair[1]}")

    if mismatches:
        for mismatch in mismatches:
            logging.info("Mismatch found:")
            logging.info(f"Video 1: {mismatch['video1']['path']}, Reported: {mismatch['video1']['reported']}, Manual: {mismatch['video1']['manual']}")
            logging.info(f"Video 2: {mismatch['video2']['path']}, Reported: {mismatch['video2']['reported']}, Manual: {mismatch['video2']['manual']}")
    else:
        logging.info("No mismatches found.")

if __name__ == "__main__":
    directory1_path ="/storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize3"
    directory2_path = "/storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS3_vids_fixed"
    process_and_compare(directory1_path, directory2_path)

### Frame number count between manual (fixed video) vs CV2 (fixed video) vs CV2 count (original video) for PS4

In [ ]:
import cv2
import os
import concurrent.futures
import logging

# Setup basic configuration for logging
logging.basicConfig(filename='video_processing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def count_frames(video_path):
    """
    Count frames manually and compare with cv2's reported frame count.
    Returns the video path, reported count, and manually counted frames.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Failed to open {video_path}")
        return None, None, None
    
    reported_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    manual_frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        manual_frame_count += 1

    cap.release()
    return video_path, reported_frame_count, manual_frame_count

def process_video_pair(video1_path, video2_path):
    """
    Process a pair of video files and compare frame counts.
    """
    path1, reported1, manual1 = count_frames(video1_path)
    path2, reported2, manual2 = count_frames(video2_path)

    # Compare and collect mismatches
    if reported1 != manual1 or reported2 != manual2 or manual1 != manual2:
        return {
            'video1': {'path': path1, 'reported': reported1, 'manual': manual1},
            'video2': {'path': path2, 'reported': reported2, 'manual': manual2}
        }
    return None

def process_and_compare(directory1, directory2):
    """
    Process video files in both directories and compare frame counts.
    """
    video_files1 = [f for f in os.listdir(directory1) if f.endswith(('.avi', '.mp4', '.mov'))]
    video_files2 = [f for f in os.listdir(directory2) if f.endswith('.mp4')]

    mismatches = []
    video_pairs = []

    for video_file in video_files1:
        base_filename = video_file.rsplit('.', 1)[0]  # Remove extension for matching
        matching_file = base_filename + '_fixed.mp4'  # Construct the fixed filename

        if matching_file in video_files2:
            video_pairs.append((os.path.join(directory1, video_file),
                                os.path.join(directory2, matching_file)))

    # Process video pairs in parallel using ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_video_pair, *pair): pair for pair in video_pairs}
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                mismatches.append(result)
            # Log progress
            video_pair = futures[future]
            logging.info(f"Processed pair: {video_pair[0]}, {video_pair[1]}")

    if mismatches:
        for mismatch in mismatches:
            logging.info("Mismatch found:")
            logging.info(f"Video 1: {mismatch['video1']['path']}, Reported: {mismatch['video1']['reported']}, Manual: {mismatch['video1']['manual']}")
            logging.info(f"Video 2: {mismatch['video2']['path']}, Reported: {mismatch['video2']['reported']}, Manual: {mismatch['video2']['manual']}")
    else:
        logging.info("No mismatches found.")

if __name__ == "__main__":
    directory1_path ="/storage/group/klk37/default/homebytes/video/fbs/PS_vids_original/PortionSize4"
    directory2_path = "/storage/group/klk37/default/homebytes/video/fbs/PS_all_vids_30fps_fixed/PS4_vids_fixed"
    process_and_compare(directory1_path, directory2_path)